# Clustering Multiple Features

**Project Goal**: Build a clustering model for multiple features. 

Specific Goals:

- Determine the highest variance features in the dataset - variance and trimmed variance.
- Build unsupervised model to divide households into groups.
- Visualize clsuters using principal component analysis (PCA) - dimensionality reduction.

In [12]:
import pandas as pd 
import plotly.express as px 
from scipy.stats.mstats import trimmed_var
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA 
from sklearn.metrics import silhouette_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.utils.validation import check_is_fitted

### Prepare Data

Import

- The goal here is to create a wrangle function that eturns a DataFrame of households whose net worth is less than $2 million and that have been turned down for credit or feared being denied credit in the past 5 years (see "TURNFEAR").

In [13]:
def wrangle(filepath):
    df=pd.read_csv(filepath)
    mask = df["TURNFEAR"] == 1
    df_fear = df[mask]
    
    return df